## Send Sensor Telemtry to RTI
Stream data from the SensorData.csv file to RTI for processing.

In [ ]:
# Install the required package
%pip install azure-eventhub

In [ ]:
# import required libraries
import random
import time
import json
import os
from azure.eventhub import EventHubConsumerClient, EventHubProducerClient,EventData
import csv
import requests
import time

In [ ]:
# DELETE LATER - THIS IS NOT NEEDED
#from pyspark.sql import SparkSession
#
## Create a Spark session
#spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

In [ ]:
# Test Cell to ensure the file is accessible and has data - Optional to run.

# Define the path to the CSV file in the Lakehouse
file_path = "abfss://Zava@onelake.dfs.fabric.microsoft.com/ZavaLakehouse.Lakehouse/Files/dtb_raw/SensorData.csv"

# Load the CSV into a DataFrame
df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

# Show the first few rows
df.show()


### Start the data generation below

In [ ]:
# Step 1 Setup Connection Info
connection_string = "";
producer = EventHubProducerClient.from_connection_string(connection_string)

# Step 2: Read CSV file from Lakehouse Files

file_path = "abfss://Zava@onelake.dfs.fabric.microsoft.com/ZavaLakehouse.Lakehouse/Files/dtb_raw/SensorData.csv"
df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

# Step 2: Convert DataFrame to JSON rows
json_rows = df.toJSON().collect()

In [6]:
# Step 4: Send 10 rows every 1 second and loop back to the beginning
with producer:
   while True:
     for i in range(0, len(json_rows), 10):
          batch = json_rows[i:i+10]
          event_data_batch = [EventData(row) for row in batch]
          producer.send_batch(event_data_batch)
          # print(f"Sent batch {i//10 + 1}")
          time.sleep(1)